In [2]:
import numpy as np
import pandas as pd

from finrock.data_feeder import PdDataFeeder
from finrock.trading_env import TradingEnv
from finrock.render import PygameRender
from finrock.scalers import ZScoreScaler
from finrock.reward import AccountValueChangeReward
from finrock.indicators import BolingerBands, SMA, RSI, PSAR, MACD
from finrock.metrics import DifferentActions, AccountValue, MaxDrawdown, SharpeRatio

# The following allows to save plots in SVG format.
import matplotlib_inline
%matplotlib inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

In [4]:
df = pd.read_csv('Datasets/random_sinusoid.csv')

pd_data_feeder = PdDataFeeder(
    df = df,
    indicators = [
        BolingerBands(data=df, period=20, std=2),
        RSI(data=df, period=14),
        PSAR(data=df),
        MACD(data=df),
        SMA(data=df, period=7),
    ]
)

env = TradingEnv(
    data_feeder = pd_data_feeder,
    output_transformer = ZScoreScaler(),
    initial_balance = 1000.0,
    max_episode_steps = 1000,
    window_size = 50,
    reward_function = AccountValueChangeReward(),
    metrics = [
        DifferentActions(),
        AccountValue(),
        MaxDrawdown(),
        SharpeRatio(),
    ]
)
action_space = env.action_space
input_shape = env.observation_space.shape

env.save_config()

pygameRender = PygameRender(frame_rate=60, screen_width=640, screen_height=480)

state, info = env.reset()
pygameRender.render(info)
rewards = 0.0
while True:
    # simulate model prediction, now use random action
    action = np.random.randint(0, action_space)

    state, reward, terminated, truncated, info = env.step(action)
    rewards += reward
    pygameRender.render(info)

    if terminated or truncated:
        print(info['states'][-1].account_value, rewards)
        rewards = 0.0
        state, info = env.reset()
        pygameRender.reset()


Paused
761.4375801174857 0.20117572876855178
3083.7471558638713 0.7637444859412226
848.1200198320481 0.36923351317184977
866.010092642239 0.41580585709683693


KeyboardInterrupt: 